# Interactive plot with a slider

## Setup and data source

Our data source are stats of the first two Pokémon Generations!

In addition to previous examples, we want to add small images of each Pokémon as Tooltips. The images are saved in a cloud in order to access it. The corresponding ULRs are a part of the dataframe

In [ ]:
### import libraries
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource

### read in data and construct column data source
df = pd.read_csv('data/pokemon_250_with_images.csv')

datasource = ColumnDataSource(data={
    "name" : df["Name"],
    "main_type" : df["Type 1"],
    "overall" : df["Total"],
    "health" : df["HP"],
    "attack" : df["Attack"],
    "defense" : df["Defense"],
    "speed" : df["Speed"],
    "imgs" : df["image_urls"]
})

## Basic code for bokeh plot

In order to access the image URLs we need to write a little html/CSS code for our tooltips
##### bokeh tutorial for this (https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#custom-tooltip)

In [ ]:
### import libraries
from bokeh.plotting import figure
from bokeh.models import HoverTool

### html code to access Pokémon images
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@imgs" height="100" alt="@imgs" width="100"
                border="2"
            ></img>
        </div>
        <div>
                <span style="font-size: 18px; font-weight: bold;">@name</span>
        </div>
        <div>
                <span style="font-size: 14px;">Overall Strength - @overall</span>
        </div>
        <div>
                <span style="font-size: 14px;">Overall Health Points - @health</span>
        </div>
"""

### define tools and build plot
toolbox = ["pan,wheel_zoom,box_zoom,reset,save,crosshair"]

plot = figure(title="Pokémon dataset", x_axis_label ="Attack", y_axis_label ="Defense",
              plot_width=800, plot_height=800, tooltips=TOOLTIPS, tools=toolbox, toolbar_location="below")

plot.circle("attack", "defense", size=10, source=datasource)

## Build bokeh app

Now we want to add all previous examples into one dashboard

In [ ]:
### import libraries
from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import TextInput
from bokeh.models.widgets import RangeSlider, RadioButtonGroup, CheckboxGroup


### build all widgets (from previous examples), we want to incorporate
hp_slider = RangeSlider(start = 10, end = 255, step = 1, value = (10,255), title = 'Pokemon Health')

overall_slider = RangeSlider(start = 180, end = 680, step = 1, value = (180,680), title = 'Pokemon Overall Strength')

main_types = df["Type 1"].unique().tolist()
types_checkbox = CheckboxGroup(labels=main_types, active = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

pokemon_textbox = TextInput(title="Name or substring of Pokemon")

datasource_choice = RadioButtonGroup(labels=["Generation 1", "Generation 2", "Generation 1 & 2"], active=2)


##### for more structure build two functions
### one which select the new data source based on user input ("select_df()")
### and one which updates the plot ("update_plot()")

### build new dataframe, based on user input and return this dataframe
def select_df():
    
    lower_boundary_hp = hp_slider.value[0]
    upper_boundary_hp = hp_slider.value[1]
    
    lower_boundary_overall = overall_slider.value[0]
    upper_boundary_overall = overall_slider.value[1]
    
    selected_types = [types_checkbox.labels[i] for i in types_checkbox.active]
    
    textbox_input = pokemon_textbox.value.strip()
    
    datasource_selection = datasource_choice.active
    
    if datasource_selection == 0:
           
        new_df = df[
            (df["HP"] >= lower_boundary_hp) & 
            (df["HP"] <= upper_boundary_hp) &
            (df["Total"] >= lower_boundary_overall) &
            (df["Total"] <= upper_boundary_overall) &
            (df["Type 1"].isin(selected_types)) &
            (df["Name"].str.contains(textbox_input)) &
            (df["Generation"]==1)
        ]
    
    elif datasource_selection == 1:
        
        new_df = df[
            (df["HP"] >= lower_boundary_hp) & 
            (df["HP"] <= upper_boundary_hp) &
            (df["Total"] >= lower_boundary_overall) &
            (df["Total"] <= upper_boundary_overall) &
            (df["Type 1"].isin(selected_types)) &
            (df["Name"].str.contains(textbox_input)) &
            (df["Generation"]==2)
        ]
    
    else:
    
        new_df = df[
            (df["HP"] >= lower_boundary_hp) & 
            (df["HP"] <= upper_boundary_hp) &
            (df["Total"] >= lower_boundary_overall) &
            (df["Total"] <= upper_boundary_overall) &
            (df["Type 1"].isin(selected_types)) &
            (df["Name"].str.contains(textbox_input))
        ]
        
    
    return new_df

### execute function for new dataframe and then update the plot with it
def update_plot():
    
    df = select_df()
    
    datasource.data = dict(
        name = df["Name"],
        main_type = df["Type 1"],
        overall = df["Total"],
        health = df["HP"],
        attack = df["Attack"],
        defense = df["Defense"],
        speed = df["Speed"],
        imgs = df["image_urls"]
    )

### when user gives input to widgets, simultaneously execute updating function for all widgets
### we have to do this separately for widgets, which rely on value input and which rely on activation input
value_changes = [hp_slider, overall_slider, pokemon_textbox]
for change in value_changes:
    change.on_change('value', lambda attr, old, new: update_plot())

active_changes = [types_checkbox, datasource_choice]
for change in active_changes:
    change.on_change('active', lambda attr, old, new: update_plot())

### add all widgets and the plot to layout
layout = row(column(datasource_choice, hp_slider, overall_slider, types_checkbox, pokemon_textbox), plot)
curdoc().add_root(layout)

### in order to show app in your browser, use command line and type "bokeh serve your_app_name.ipynb --show"